In [1]:
import numpy as np
import pandas as pd
import requests
import sqlite3
from pathlib import Path


In [2]:
polling = pd.read_csv("../president_primary_polls.csv")


In [23]:
np.sort

<function numpy.sort(a, axis=-1, kind=None, order=None)>

In [22]:
polling["state"].unique() list(map(lambda x: "National" if x != x else x, polling["state"].unique()))

SyntaxError: invalid syntax (<ipython-input-22-53b3fbddf096>, line 1)

In [19]:
polling["state"][0] == polling["state"][0]

False

In [14]:
url = 'https://projects.fivethirtyeight.com/polls-page/president_primary_polls.csv'
r = requests.get(url, allow_redirects=True)

In [ ]:
# r.content
from io import StringIO
temp = pd.read_csv(StringIO(str(r.content)))

In [ ]:
temp

In [9]:
polling_subset = polling[polling["party"] == "DEM"]
polling["create_date"] = pd.to_datetime(polling["created_at"], infer_datetime_format=True)
polling["create_date"]
polling["create_week"] = (polling['create_date'] - pd.to_timedelta(polling['create_date'].dt.dayofweek, unit='d')
                          + np.timedelta64(7, 'D')).dt.normalize()                                                             
polling_subset = polling[polling["party"] == "DEM"]
df_pivot = polling[polling["party"] == "DEM"].pivot_table(values="pct", index="create_week", columns="answer",
                                                          aggfunc=np.mean).fillna(0).iloc[-7:]
candidates = polling_subset["answer"].unique()


In [12]:
polling_subset[["answer", "candidate_id"]]

,answer,candidate_id
3,Booker,13287
4,Biden,13256
5,Sanders,13257
6,Warren,13258
7,Buttigieg,13345
...,...,...
13754,Sanders,13257
13755,Booker,13287
13756,Harris,13288
13757,Bloomberg,13289


In [7]:
can_selected = []
for can in candidates:
    if (df_pivot[can] == 0).mean() < 0.5 and df_pivot[can].mean() > 0.1:
        print(can)
        can_selected.append(can)


Booker
Biden
Sanders
Warren
Buttigieg
Harris
Yang
Bloomberg
Castro
Klobuchar
Gabbard
Steyer
Delaney
Sestak
Williamson
Bullock
Bennet
O'Rourke
Ryan


In [6]:
print('Beginning file download with requests')

for can in can_selected:
    url = 'https://api.gdeltproject.org/api/v2/tv/tv?query="' + can + '"%20market:%22National%22&mode=timelinevol&format' \
                                                                      '=csv&datanorm=perc&dateres=week&timelinesmooth=0' \
                                                                      '&datacomb=sep' \
                                                                      '&last24=yes&timezoom=yes '
    r = requests.get(url)

    with open('../candidates_coverage/' + can + '.csv', 'wb') as f:
        f.write(r.content)

    print(url)
    print(r.status_code)
    print(r.headers['content-type'])
    print(r.encoding)


Beginning file download with requests


FileNotFoundError: [Errno 2] No such file or directory: '../candidates_coverage/Booker.csv'

In [73]:
source_files = sorted(Path('../candidates_coverage').glob('*.csv'))

dataframes = []
for file in source_files:
    df = pd.read_csv(file)
    df['Candidate'] = file.name.replace(".csv", "")
    dataframes.append(df)

media_coverage = pd.concat(dataframes, sort=False)
media_coverage.rename(columns={(media_coverage.columns[0]): 'Date'}, inplace=True)
media_coverage['Date'] = pd.to_datetime(media_coverage['Date'])
media_coverage.to_csv('C:\\Users\\User\\PycharmProjects\\WebDB\\polls\\media_coverage.csv')


In [ ]:
polling_means = polling_subset.groupby(['answer', 'create_week'])['pct'].mean().reset_index()
conn = sqlite3.connect(':memory:')
media_coverage.to_sql('media_coverage', conn, index=False)
polling_means.to_sql('polling_means', conn, index=False)
qry = '''
    SELECT *
    FROM
        polling_means JOIN media_coverage ON
        Candidate = answer
        AND create_week BETWEEN Date AND date(Date, '+7 day')
    '''
corr_data = pd.read_sql_query(qry, conn)
# corr_data.to_csv('../polls/corr_data.csv')
